# Dependence of the pH on the added carbon dioxide amount

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Mar 31th, 2022</p>

This tutorial demonstrates the modeling of the pH dependence in the seawater on the CO<sub>2</sub>(g) amount added, which causes the so-called ocean acidification. This phenomenon remains one of the main problems of human activities, namely the burning of fossil fuels. As carbon dioxide in the atmosphere increases, so does the amount of carbon dioxide absorbed by the ocean. This leads to a series of chemical reactions in seawater that negatively impact the ocean and submerged species. In particular, this leads to decreased production of shells of bivalves and other aquatic organisms with calcium carbonate shells, as well as some other physiological challenges for marine organisms.

|![The CO2 cycle between the atmosphere and the ocean](../../images/applications/co2-pump-ocean-acidification.png)|
|:--:|
|The CO2 cycle between the atmosphere and the ocean, Source: wikipedia.org|

We start by defining the chemical system containing aqueous and gaseous phases.  

In [1]:
from reaktoro import *
import pandas as pd

db = SupcrtDatabase("supcrtbl")

# Create an aqueous phase automatically selecting all species with provided elements
aqueousphase = AqueousPhase(speciate("H O C Ca Mg K Cl Na S N"))
aqueousphase.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2"),
))

# Create a gaseous phase
gaseousphase = GaseousPhase("CO2(g)")
gaseousphase.setActivityModel(ActivityModelPengRobinson())

# Create the chemical system
system = ChemicalSystem(db, aqueousphase, gaseousphase)

# Create the equilibrium solver
solver = EquilibriumSolver(system)

Next, we initialize the chemical state corresponding to the seawater content, equilibrate it, and evaluate the pH level obtained after equilibration:

In [2]:
state = ChemicalState(system)
state.setTemperature(25, "celsius")
state.setPressure(1.0, "bar")
state.add("H2O(aq)",      1.0, "kg")
state.add("Ca+2" ,   412.3, "mg")
state.add("Mg+2" ,  1290.0, "mg")
state.add("Na+"  , 10768.0, "mg")
state.add("K+"   ,   399.1, "mg")
state.add("Cl-"  , 19353.0, "mg")
state.add("HCO3-",   141.7, "mg")
state.add("SO4-2",  2712.0, "mg")

solver.solve(state)

aprops = AqueousProps(state)
print("pH of seawater = ", float(aprops.pH()))

pH of seawater =  7.6990617042331415


Finally, we define the auxiliary variables with the initial values of the CO<sub>2</sub> amount and its increment. We run the loop with `nsteps` steps adding CO<sub>2</sub> into seawater and re-equilibrate it again. We collect the amount of added carbon dioxide and the corresponding pH value in `pandas.DataFrame`.

In [3]:
co2_0 = 0.0
co2_delta = 0.1
nsteps = 50

df = pd.DataFrame(columns=["amountCO2", "pH"])
df.loc[len(df)] = [co2_0, float(aprops.pH())]

for i in range(nsteps):

    # Add more CO2 to the problem
    state.add("CO2(g)", co2_delta, "mmol")

    # Equilibrate state with updated problem
    solver.solve(state)

    # Update aqueous properties
    aprops.update(state)

    # Update CO2 amount
    co2_0 += co2_delta

    # Append new calculated value to the dataframe
    df.loc[len(df)] = [co2_0, float(aprops.pH())]

We use the [bokeh](https://bokeh.org/) plotting library, to plot pH as a function of the CO<sub>2</sub> amount added into the seawater.

In [4]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Legend
from bokeh.io import output_notebook
output_notebook()

hovertool = HoverTool()
hovertool.tooltips = [("pH", "@pH"), ("amount(CO2)", "@amountCO2 mmol")]

p = figure(
    title="PH DEPENDENCE ON AMOUNT OF ADDED CO2 TO THE SEAWATER",
    x_axis_label=r'CO2 AMOUNT [MMOL]',
    y_axis_label='PH [-]',
    sizing_mode="scale_width",
    plot_height=200)

p.add_tools(hovertool)

p.line("amountCO2", "pH", line_width=3, line_cap="round", line_color='indigo', source=df)

show(p)

Loading BokehJS ...

As expected, the pH decreases as we continue to add CO<sub>2</sub> into the seawater, the solution gradually becomes more acidic.